# Проект для «Викишоп»

**Проект Поиск токсичных комментариев**

**Задача:** Обучить модель классифицировать комментарии на позитивные и негативные

**Цель:** Построить модель со значением метрики качества F1 не меньше 0.75.

**Данные:** В вашем распоряжении набор данных с разметкой о токсичности правок.

**Действия:** проверить данные, удалить все лишние символы из текста, осуществить ламматизацию текста, убрать стоп-слова и построить несколько моделей.

## Подготовка

In [1]:
# Загрузим библиотеки

import pandas as pd
from pymystem3 import Mystem

import re
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from nltk.corpus import stopwords as nltk_stopwords
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# откроем данные для ознакомления

data = pd.read_csv('/datasets/toxic_comments.csv')
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
# получим информацию о данных

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
# колонка Unnamed неинформативная, поэтому удалим ее

data = data.drop('Unnamed: 0', axis=1)

In [5]:
# в колонке Toxic содерится только бинарное значение, как и должно быть 

data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [6]:
# проверим наличие дубликатов

data.duplicated().sum()

0

In [14]:
# напишем функции лемматизации и удаления лишних символов

def clear_text(text):
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    text = text.split()
    text = " ".join(text)

    return text



In [15]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

m = WordNetLemmatizer()

def lemmatize(text):
    text = nltk.word_tokenize(text)
    
    return ' '.join([m.lemmatize(w, get_wordnet_pos(w)) for w in text])

In [16]:
%%time

data['text'] = data['text'].apply(clear_text)
data['lemm_text'] = data['text'].apply(lemmatize)

CPU times: user 17min 11s, sys: 1min 39s, total: 18min 51s
Wall time: 18min 52s


In [18]:
# после создания лемматизированного текста. Первоначальный текст можно удалить из признаков

data = data.drop(['text'], axis=1)

## Вывод:
Предоставленные данные полностью заполнены,не имеют пропусков и дубликатов. В данных была неинформационная колонка, которая дублирует индекс, ее целесообразно было удалить из данных. В тексте были удалены лишние символы и создан лемматизированный текст для дальнейшей обработки.

## Обучение

### Подготовка данных к обучению

In [19]:
corpus = data['lemm_text'].values

In [20]:
# разобьем данные на признак и целевой признак, а затем разобьем данные на обучающую и тестовую выборки

feature = corpus
target = data['toxic'].values

feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=.2, random_state=12345)


In [21]:
# проверим корректность разбивки выборок

print(feature_train.shape)
print(target_train.shape)
print(feature_test.shape)
print(target_test.shape)

(127433,)
(127433,)
(31859,)
(31859,)


In [22]:
%%time

# удалим неинформативные слова из текста

stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words = stopwords)
feature_train = count_tf_idf.fit_transform(feature_train)
feature_test = count_tf_idf.transform(feature_test)

CPU times: user 6.43 s, sys: 44 ms, total: 6.48 s
Wall time: 6.48 s


In [23]:
print(feature_train.shape)
print(target_train.shape)
print(feature_test.shape)
print(target_test.shape)

(127433, 139501)
(127433,)
(31859, 139501)
(31859,)


In [24]:
# напишем функцию модели

def model_look(model, parameters, feature, target):
    model_grid = GridSearchCV(model, parameters, cv=3, scoring='f1', verbose=5)
    model_grid.fit(feature, target)
    return model_grid.best_score_, model_grid.best_params_

### Линейная регрессия

In [25]:
%%time

# найдем наилучшие параметры для модели при помощи GridSearchCV

parameters_lg = {'C': range(22, 26, 2), 'max_iter': range(8, 10, 1)}
f1, best_parameters_lg = model_look(LogisticRegression(random_state=12345, solver='liblinear'), parameters_lg, feature_train, target_train)
print('Лучшие параметры модели линейной регрессии:', best_parameters_lg)
print('F1 линейной регреcсии:', f1)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END ...............................C=22, max_iter=8; total time=   5.2s
[CV 2/3] END ...............................C=22, max_iter=8; total time=   5.2s
[CV 3/3] END ...............................C=22, max_iter=8; total time=   5.1s
[CV 1/3] END ...............................C=22, max_iter=9; total time=   6.1s
[CV 2/3] END ...............................C=22, max_iter=9; total time=   6.0s
[CV 3/3] END ...............................C=22, max_iter=9; total time=   5.9s
[CV 1/3] END ...............................C=24, max_iter=8; total time=   5.1s
[CV 2/3] END ...............................C=24, max_iter=8; total time=   5.2s
[CV 3/3] END ...............................C=24, max_iter=8; total time=   5.1s
[CV 1/3] END ...............................C=24, max_iter=9; total time=   5.9s
[CV 2/3] END ...............................C=24, max_iter=9; total time=   5.7s
[CV 3/3] END ...............................C=24,

In [27]:
# создадим датасет с полученными данными 

results = pd.DataFrame ({'model':['LogisticRegression'], 'best_parameters_train':[best_parameters_lg], 'f1_train':[f1]})
results

,model,best_parameters_train,f1_train
0,LogisticRegression,"{'C': 22, 'max_iter': 8}",0.777892


### Случайный лес

In [24]:
%%time

# # найдем наилучшие параметры для модели при помощи GridSearchCV
# прошлась диапазонами по max_depth, но ни один результат не дал необходимого результата

parameters_rf = {'max_depth': range(212, 213, 1), 'n_estimators': range(2, 6, 2)}
f1, best_parameters_rf = model_look(RandomForestClassifier(random_state=123), parameters_rf, feature_train, target_train)
print('Лучшие параметры модели случайного леса:', best_parameters_rf)
print('F1 модели Случайный лес:', f1)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END ..................max_depth=212, n_estimators=2; total time=  10.1s
[CV 2/3] END ..................max_depth=212, n_estimators=2; total time=   9.9s
[CV 3/3] END ..................max_depth=212, n_estimators=2; total time=  10.0s
[CV 1/3] END ..................max_depth=212, n_estimators=4; total time=  20.1s
[CV 2/3] END ..................max_depth=212, n_estimators=4; total time=  19.3s
[CV 3/3] END ..................max_depth=212, n_estimators=4; total time=  22.0s
Лучшие параметры модели случайного леса: {'max_depth': 212, 'n_estimators': 4}
F1 модели Случайный лес: 0.5733774541471263
CPU times: user 1min 46s, sys: 454 ms, total: 1min 46s
Wall time: 1min 47s


In [26]:
# внесем полученные данные в датасет

results = results.append({'model':'RandomForestClassifier', 'best_parameters_train':best_parameters_rf, 'f1_train':f1}, ignore_index=True)
results

,model,best_parameters_train,f1_train
0,LogisticRegression,"{'C': 24, 'max_iter': 8}",0.762972
1,RandomForestClassifier,"{'max_depth': 212, 'n_estimators': 4}",0.573377


### CatBoostClassifier

In [ ]:
# закомментировала модель, т.к. ее обработка идет несколько часов

"""%%time 

# найдем наилучшие параметры для модели при помощи GridSearchCV

parameters_cat = {'iterations': range(1000, 1010, 3)}
f1, best_parameters_cat = model_look (CatBoostClassifier(random_state=123), parameters_cat, feature_train, target_train)
print('Лучшие параметры модели:', best_parameters_cat)
print('F1 модели CatBoostClassifier:', f1)"""

In [ ]:
"""results = results.append({'model':'CatBoostClassifier', 'best_parameters_train':best_parameters_cat, 'f1_train':f1}, ignore_index=True)
results"""

### Проверка на тестовой выборке

In [28]:
# Наилучший результат дала модель Линейная регрессия с результатом F1 = 0,76. Проверим результаты на тестовой выборке

model_lg = LogisticRegression(random_state=12345, solver='liblinear', C = 22,  max_iter = 8)
model_lg.fit(feature_train, target_train)
predictions = model_lg.predict(feature_test)
f1 = f1_score(target_test, predictions)

print('F1:', f1)

F1: 0.7780821917808219


## Выводы

Предоставленные данные полностью заполнены, не имеют пропусков и дубликатов. В данных была неинформационная колонка, которая дублирует индекс, ее целесообразно было удалить из данных. В тексте были удалены лишние символы и создан лемматизированный текст для дальнейшей обработки.
В ходе анализа были созданы 3 модели: Линейная регрессия, Случайный лес и CatBoostClassifier. Наилучший результат на обучающей выборке показала Линейная регрессиия с F1 = 0.76. Финишный этап проверки модели Линейной регрессии на тестовой выборке показал F1 = 0,78, что соответствует требованиям задания, поэтому модель Линейной регрессии можно считать рабочей для данного проекта.